In [1]:
import sys
from pathlib import Path

def add_project_root(levels_up: int = 1):
    """Adds the project root to sys.path for Jupyter or scripts."""
    root = Path().resolve()
    for _ in range(levels_up):
        root = root.parent
    if str(root) not in sys.path:
        sys.path.append(str(root))
add_project_root() 

In [2]:
import pandas as pd
from transformers import pipeline
import evaluate

from modules.pipeline import complete_pipeline
from modules.summarizer import Summarizer
from modules.SummaryManager import EvalSummaryManager
from modules.summary_enhancer import enhancer
from modules.data_handler import load_discharges, load_patient_data
from modules.context_handler import context_builder_text, context_builder_json, build_discharge_windows, get_patient_window_data

/home/sandor/.conda/envs/verdi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sandor/.conda/envs/verdi/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/sandor/.conda/envs/verdi/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
DECOMPOSITION_PROMPT_TEMPLATE = '''

You are a medical information analyst. Your task is to parse the entire medical summary below and decompose it into a complete list of fine-grained, verifiable claims. These claims will be used to fact-check the summary against a full patient record.

Each claim must be a single, self-contained medical fact, finding, instruction, or patient-reported statement.

**Extraction Rules:**
1.  **Split Lists:** Decompose lists of conditions, symptoms, findings, or plans into separate claims (e.g., "history of A, B, and C" becomes three separate claims).
2.  **Keep Instructions Intact:** A single medication (e.g., "Furosemide 40mg daily") is one complete claim. Do not split the drug from its dosage.
3.  **Be Self-Contained:** Resolve all pronouns (e.g., "She," "He") to "The patient."
4.  **Extract All Details:** Capture demographics, past medical history (PMH), symptoms, lab results, imaging findings, diagnoses, medications, and follow-up plans.
5.  **Output Format:** Provide the output as a simple list of claims one in each line.

---
### Example 1

**Full Summary:**
The patient, a female with a history of HCV cirrhosis with ascites, HIV on ART, IV drug use, COPD, bipolar disorder, and PTSD, presented with worsening abdominal distension and pain. She had discontinued her Lasix and Spironolactone prior to admission due to feeling that they weren't effective and wanting to avoid "chemicals."During hospitalization, the patient's ascites was managed with diuretics (Furosemide 40mg and Spironolactone 50mg daily). Paracentesis was attempted in the ED but was unsuccessful. Lab results showed elevated liver enzymes (ALT, AST, Alk Phos, Total Bili), and a low platelet count. A CXR showed no acute cardiopulmonary process. Ultrasound revealed liver nodularity, signs of portal hypertension (ascites and splenomegaly), and cholelithiasis. The patient was scheduled for follow-up with her current PCP, a new PCP (Dr. ___), and the Liver Clinic for EGD and ___. She was discharged home in a clear and coherent mental state, alert, interactive, and ambulatory, with a diagnosis of ascites from portal hypertension. She was instructed to continue taking Furosemide 40mg and Spironolactone 50mg daily, adhere to a low-salt diet, and follow up with the Liver Clinic.


**Claims:**
The patient is female.

The patient has a history of HCV cirrhosis with ascites.

The patient has a history of HIV on ART.

The patient has a history of IV drug use.

The patient has a history of COPD.

The patient has a history of bipolar disorder.

The patient has a history of PTSD.

The patient presented with worsening abdominal distension.

The patient presented with worsening abdominal pain.

The patient had discontinued her Lasix prior to admission.

The patient had discontinued her Spironolactone prior to admission.

The patient's reason for discontinuing was feeling [the drugs] weren't effective.

The patient's reason for discontinuing was wanting to avoid "chemicals."

The patient's ascites was managed with Furosemide 40mg daily during hospitalization.

The patient's ascites was managed with Spironolactone 50mg daily during hospitalization.

Paracentesis was attempted in the ED.

The paracentesis attempt in the ED was unsuccessful.

Lab results showed elevated liver enzymes (ALT, AST, Alk Phos, Total Bili).

Lab results showed a low platelet count.

A CXR showed no acute cardiopulmonary process.

Ultrasound revealed liver nodularity.

Ultrasound revealed signs of portal hypertension (ascites and splenomegaly).

Ultrasound revealed cholelithiasis.

The patient was scheduled for follow-up with her current PCP.

The patient was scheduled for follow-up with a new PCP (Dr. ___).

The patient was scheduled for follow-up with the Liver Clinic for EGD.

The patient was scheduled for follow-up with the Liver Clinic for ___.

The patient was discharged home.

The patient's mental state at discharge was clear and coherent.

The patient's status at discharge was alert.

The patient's status at discharge was interactive.

The patient's status at discharge was ambulatory.

The patient's diagnosis was ascites from portal hypertension.

The patient was instructed to continue taking Furosemide 40mg daily.

The patient was instructed to continue taking Spironolactone 50mg daily.

The patient was instructed to adhere to a low-salt diet.

The patient was instructed to follow up with the Liver Clinic.

---
### New summary (YOUR TASK)

**Full Summary:**
{summary_text}
'''


In [ ]:

# Preload the NLI model once (outside function for efficiency)
nli = pipeline("text-classification", model="facebook/bart-large-mnli")


def decomp_judge_subpipeline():
    discharges = load_discharges()
    patients_data = load_patient_data()
    discharge_windows = build_discharge_windows(discharges)

    summarizer = Summarizer(model='gemini')

    results = []
    idx=0

    for _, case in discharges.iterrows():
        subject_id = case['subject_id']
        hadm_id = case['hadm_id']
        original_text = case['text']


        # Retrieve contextual data
        context_df = get_patient_window_data(
            subject_id=subject_id,
            hadm_id=hadm_id,
            windows_dict=discharge_windows,
            patient_df=patients_data,
            time_col='starttime'
        )
        context_json = context_builder_json(context_df)
        context_textual = context_builder_text(context_json)


        ### Summarization -------------------------------------------------------------
        summary_response = summarizer.summ(original_text)


        ### Decomposition -------------------------------------------------------------
        decomposition_prompt = DECOMPOSITION_PROMPT_TEMPLATE.format(
            summary_text=summary_response
        )
        decomp_response = summarizer.model.generate_content(decomposition_prompt).text


        ### Judging -------------------------------------------------------------------
        claims = [c.strip() for c in decomp_response.splitlines() if c.strip()]
        judged_claims_list = []
        judged_claims_text_list = []

        # Chunking with overlap
        premise = f"Discharge:\n{original_text}\nPatient context:\n{context_textual}"

        chunk_size = 2000          # text length per chunk
        overlap = 200              # number of overlapping characters
        premise_chunks = [
            premise[i : i + chunk_size]
            for i in range(0, len(premise), chunk_size - overlap)
        ]

        # Run for each 
        for claim in claims:

            best_label, best_score = None, 0.0
            for chunk in premise_chunks:
                result = nli(f"{chunk} </s> {claim}")
                label, score = result[0]["label"], result[0]["score"]

                if score > best_score:
                    best_label, best_score = label, score

                # Early stop
                if label == "ENTAILMENT" and score > 0.85:
                    break

            judged_claim = f"{claim}-{best_label}-{best_score:.3f}"
            judged_claims_list.append(judged_claim)

            judged_claim_text = f"{claim} - {best_label} - {best_score:.3f}"
            judged_claims_text_list.append(judged_claim_text)

        judged_claims_text = "\n".join(judged_claims_list)
        


        print('discharge: \n', original_text, '\nsummary:\n', summary_response,'\ndecomp:\n', decomp_response,'\ncontext:\n', context_textual, '\n*****\nJUDGED CLAIMS\n', judged_claims_text,'\n\n',)

        ## Claim Correction --------------------------------------------------------------------------------------


        ## Text Compiler ------------------------------------------------------------------------------------------
        prompt = '''
            You are a precise and concise medical writer.
            Your task is to merge and rewrite the following factual sentences into a single, well-structured text.

            Guidelines:
             - Only combine sentences if their meanings are fully equivalent or trivially overlapping.
             - Maintain all factual information — do not omit or change any details.
             - If there is any uncertainty about whether two sentences mean the same thing, keep both.
             - Preserve the order of the claims
             - Use a natural narrative tone, combining ideas logically (e.g., history -> reason for admission -> findings).
             - If multiple facts are related, integrate them into one flowing sentence.
             - Preserve medical accuracy, structure, and readability.
             - Do not speculate or interpret beyond what is written.


            Example 1:
            Input:
            The patient reported self-discontinuing her diuretics.
            The patient reported self-discontinuing Lasix.
            The patient reported self-discontinuing Spironolactone.
            The patient reported not adhering to a sodium-restricted diet.
            Self-discontinuing diuretics and not adhering to a sodium-restricted diet likely contributed to the worsening ascites.


            Output:
            The patient reported self-discontinuing her diuretics, including Lasix and Spironolactone, and not adhering to a sodium-restricted diet, both of which likely contributed to her worsening ascites.


            Example 2:
            Input:
            The patient has a history of HCV cirrhosis.
            The patient has a history of HIV on ART.
            The patient has a history of COPD.
            The patient has a history of bipolar disorder.
            The patient has a history of PTSD.

            Output:
            The patient has a history of HCV cirrhosis, HIV on ART, COPD, bipolar disorder, and PTSD.

            *NOW AGGREGATE (YOUR TASK)*:
            

        '''
        prompt = prompt + decomp_response

        enhanced_summary = summarizer.model.generate_content(prompt).text

        ## Compute Metrics ---------------------------------------------------------------------------------------
        # summary_response
        # enhanced_summary

        # BERT_score

        rouge = evaluate.load('rouge')
        rouge_scores = rouge.compute(
            predictions=[summary_response, enhanced_summary],
            references=[original_text, original_text],
            use_aggregator=False
        )


        results.append({
            "subject_id": subject_id,
            "hadm_id": hadm_id,
            "original_text": original_text,
            "context_textual": context_textual,
            "base_summary": summary_response,
            "decomposed_summary_text": judged_claims_text,
            "judged_claims": judged_claims_list,
            # "nli_results": nli_results
            "rouge_base_summary": {
                "rouge1": rouge_scores["rouge1"][0],
                "rouge2": rouge_scores["rouge2"][0],
                "rougeL": rouge_scores["rougeL"][0],
                "rougeLsum": rouge_scores["rougeLsum"][0],
            },
            "rouge_enhanced_summary": {
                "rouge1": rouge_scores["rouge1"][1],
                "rouge2": rouge_scores["rouge2"][1],
                "rougeL": rouge_scores["rougeL"][1],
                "rougeLsum": rouge_scores["rougeLsum"][1],
            },
        })


        results_df = pd.DataFrame(results)
        results_df.to_csv('results_df.csv', index=False)
        print(f"Checkpoint saved ({idx}/{len(discharges)}) → {'results_df.csv'}")
        idx += 1

    results_df = pd.DataFrame(results)
    results_df.to_csv('results_df.csv', index=False)
    return results_df


In [ ]:
decomp_judge_subpipeline()

In [ ]:
summarizer = Summarizer(model='gemini')

prompt = '''
    You are a careful medical summarizer.
    Your task is to merge and unify sentences that clearly describe the same information, while keeping all distinct clinical facts intact.

    Guidelines:
        - Only combine sentences if their meanings are fully equivalent or trivially overlapping.
        - Never remove or alter any factual information.
        - If there is any uncertainty about whether two sentences mean the same thing, keep both.
        - Preserve the order of the claims
        - Preserve medical accuracy, structure, and readability.
        - Output the result as a clear, concise list of sentences that retain all original meaning.


    Example 1:
    Input:
    The patient reported self-discontinuing her diuretics.
    The patient reported self-discontinuing Lasix.
    The patient reported self-discontinuing Spironolactone.
    The patient reported not adhering to a sodium-restricted diet.
    Self-discontinuing diuretics and not adhering to a sodium-restricted diet likely contributed to the worsening ascites.
    The patient has a history of HCV cirrhosis.
    The patient has a history of HIV on ART.
    The patient has a history of COPD.
    The patient has a history of bipolar disorder.
    The patient has a history of PTSD.

    Output:
    The patient has a history of HCV cirrhosis, HIV on ART, COPD, bipolar disorder, and PTSD. She reported self-discontinuing her diuretics, including Lasix and Spironolactone, and not adhering to a sodium-restricted diet, both of which likely contributed to her worsening ascites.


    Example 2:
    Input:
    The patient is female.
    The patient has a history of HIV on HAART.
    The patient has a history of COPD.
    The patient has a history of HCV cirrhosis.
    The patient's HCV cirrhosis is complicated by ascites.
    The patient's HCV cirrhosis is complicated by hepatic encephalopathy.
    The patient was admitted due to abdominal fullness.
    The patient was admitted due to abdominal discomfort.
    The patient had been recently discharged after a paracentesis.
    The patient experienced re-accumulation of ascites.
    The patient experienced increased abdominal distention.
    The patient experienced increased abdominal pain.
    The patient experienced visual hallucinations.
    The patient experienced forgetfulness.
    The patient's visual hallucinations are suggestive of hepatic encephalopathy.
    Upon discharge, the patient was independently ambulatory.
    Upon discharge, the patient had clear mental status.
    The patient's primary discharge diagnosis was diuretic refractory ascites.
    The patient's secondary discharge diagnosis was HCV cirrhosis.
    The patient's secondary discharge diagnosis was HIV.
    The patient's secondary discharge diagnosis was hyponatremia.
    The patient's secondary discharge diagnosis was COPD.
    The patient was instructed to continue her low-sodium diet.
    The patient was instructed to continue her fluid restriction.
    The patient was advised to return to the emergency room for any worsening abdominal pain.
    The patient was advised to return to the emergency room for any fever.
    The patient's discharge medications included her HIV medications.
    The patient's discharge medications included lactulose.
    The patient's discharge medications included rifaximin.
    The patient's discharge medications included Bactrim.

    Output:
    The patient is a female with a history of HCV cirrhosis complicated by ascites and hepatic encephalopathy, HIV on HAART, COPD, and hyponatremia. She was admitted with abdominal fullness and discomfort following a recent discharge after paracentesis, due to re-accumulation of ascites associated with increased abdominal distention and pain. She also experienced visual hallucinations and forgetfulness, findings suggestive of hepatic encephalopathy. Upon discharge, she was independently ambulatory and had a clear mental status. Her primary discharge diagnosis was diuretic-refractory ascites, with secondary diagnoses of HCV cirrhosis, HIV, hyponatremia, and COPD. She was instructed to continue a low-sodium diet and fluid restriction, and to return to the emergency room for any worsening abdominal pain or fever. Discharge medications included her HIV regimen, lactulose, rifaximin, and Bactrim.

    *NOW AGGREGATE (YOUR TASK)*:
    

'''

claims = '''

    The patient is female.
    The patient is ___ years old.
    The patient is healthy.
    The patient was admitted to Urology.
    The patient was admitted for a right renal mass.
    The right renal mass was suspicious for Renal Cell Carcinoma (RCC).
    The right renal mass was an incidental finding on MRI.
    The patient's past medical history includes nonspecific right axis deviation.
    The patient's past medical history includes a cesarean section.
    The patient has no known drug allergies.
    The patient's diet was gradually advanced.
    The remainder of the patient's hospital course was unremarkable.
    At discharge, the patient was in stable condition.
    At discharge, the patient was ambulating independently.
    At discharge, the patient was tolerating a diet.
    At discharge, the patient was voiding without difficulty.
    At discharge, the patient's pain was well-controlled on oral medication.
    Discharge medications included hydrocodone-acetaminophen for breakthrough pain.
    Discharge medications included docusate sodium for constipation.
    The patient was discharged home.
    The patient was instructed to avoid bathing.
    The patient was instructed to avoid swimming.
    The patient was instructed to limit lifting to under 10 pounds.
    The patient was instructed to avoid driving until her follow-up.
    The patient was instructed to use Tylenol as first-line pain relief.
    The patient was advised to resume home medications.
    The patient was advised to hold NSAIDs until follow-up.
    The patient was advised to seek medical attention for fevers above 101.5 F.
    The patient was advised to seek medical attention for vomiting.
    The patient was advised to seek medical attention for worsening incision.
    A follow-up appointment with Urology in 3 weeks was scheduled.
    The patient's final diagnosis was renal cell carcinoma.

'''

prompt = prompt + claims

output = summarizer.model.generate_content(prompt).text


print(output)

In [ ]:
summarizer = Summarizer(model='gemini')



In [ ]:
pd.DataFrame(data)

In [3]:
complete_pipeline()

/home/sandor/.conda/envs/verdi/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Processing case 1 - subject 10000032, hadm 22595853
 Starting summary...
Decomposing into claims...
['The patient is female.', 'The patient has a history of HCV cirrhosis.', 'The patient has a history of HIV on ART.', 'The patient has a history of COPD.']
Processing 4 claims...
Compiling new summary
ERROR!! on case 1: sequence item 0: expected str instance, dict found

Processing case 2 - subject 10000032, hadm 22841357
 Starting summary...
Decomposing into claims...
['The patient is female.', 'The patient has a history of HIV on HAART.', 'The patient has a history of COPD.', 'The patient has a history of HCV cirrhosis complicated by ascites.']
Processing 4 claims...


KeyboardInterrupt: 